In [3]:
import cv2,os
import numpy as np
from keras.utils import np_utils
import tensorflow as tf
import keras
import imutils
import pytesseract
from keras.models import load_model
import datetime
from twilio.rest import Client

import matplotlib.pyplot as plt
%matplotlib inline

labels_dict={1:'with_mask', 0:'without_mask'}
color_dict={0:(0,255,0),1:(0,0,255)}

source=cv2.VideoCapture(0)

fourcc = cv2.VideoWriter_fourcc('X', 'V', 'I', 'D')
record = False

face_clsfr=cv2.CascadeClassifier('../data/haarcascade_frontalface_default.xml')

model = load_model('095-0.3610.model')

account_sid = 'Twilio API'
auth_token = 'Twilio API'
now = datetime.datetime.now().strftime("%d_%H-%M-%S")
while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(gray,1.3,5)  
    
    for x,y,w,h in faces:
    
        face_img=gray[y:y+w,x:x+w]
        resized=cv2.resize(face_img,(100,100))
        normalized=resized/255.0
        reshaped=np.reshape(normalized,(1,100,100,1))
        result=model.predict(reshaped)

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        
        if(labels_dict[label] == 'with_mask'):
            print("마스크 쓰고 있는 중")
        else:
            cv2.imwrite("../data/capture/" + str(now) + ".png", img)
            client = Client(account_sid, auth_token)
            message = client.messages.create(
                        to="+821021277767", 
                        from_="+19092807561",
                        body="마스크 쓰세요")
            print("마스크 안 씀 / capture or send SMS")   
        
        cv2.putText(
          img, "{}: {:.2f}%".format(labels_dict[label], np.max(result) * 100),            
          (x, y-10),
          cv2.FONT_HERSHEY_SIMPLEX,0.6,(255,255,255),2)
        
        
    now = datetime.datetime.now().strftime("%d_%H-%M-%S")
    key = cv2.waitKey(33)
    
    cv2.imshow('LIVE',img)
    
    if key == 27: # ESC
        #os.startfile("../data/capture/")
        print("종료")
        break   
    elif key == 99: # C
        print("캡쳐")
        cv2.imwrite("../data/capture/" + str(now) + ".png", img)
    elif key == 115: # S
        print("녹화 시작")
        record = True
        out = cv2.VideoWriter("../data/capture/" + str(now) + ".avi", fourcc, 20, (640, 480))
    elif key == 122: # Z
        print("녹화 중지")
        record = False
        out.release()
        
    if record == True:
        out.write(img)

source.release()

cv2.destroyAllWindows()

마스크 안 씀 / capture or send SMS
마스크 안 씀 / capture or send SMS
마스크 안 씀 / capture or send SMS
마스크 안 씀 / capture or send SMS
종료
